In [1]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import pandas as pd


In [2]:
## Utility .py file.

def getcontour(img, thr = [100,100], showcanny = False, minarea = 10000, filter = 4, draw = False):
    imggray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    imgblur = cv2.GaussianBlur(imggray, (5,5), 1)
    imgcanny = cv2.Canny(imgblur, thr[0], thr[1])
    kernel = np.ones((5,5)) 
    imgdial = cv2.dilate(imgcanny, kernel, iterations = 3)
    imgeros = cv2.erode(imgdial, kernel, iterations = 2)
    
    if showcanny:cv2.imshow('Canny', imgeros)

    contours, hierarchy = cv2.findContours(imgeros, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Constraints, for area of contour.
# Approximation of curve, that is with 4 point and closed curve for quadrilateral.
# cv2.approxPolyDP function to smooth and approximate the quadrilateral.
    finalcontour = []

    for i in contours:
        area = cv2.contourArea(i)
        if area > minarea:
            peri = cv2.arcLength(i, True)
            approx = cv2.approxPolyDP(i, 0.02*peri, True) 
            bbox = cv2.boundingRect(approx)

            if filter > 0:                 # if curve of the approximation has minimum 4 points.
                if len(approx) == filter:  # if approximation curve has minimum 4 point then we are choosing those contours curve.  
                    finalcontour.append([len(approx), area, approx, bbox, i])
            else:
                finalcontour.append([len(approx), area, approx, bbox, i])
    finalcontour = sorted(finalcontour, key = lambda finalcontour: finalcontour[1], reverse = True) ## It will become Big to small using reverse = True
    if draw:
        for con in finalcontour:
            cv2.drawContours(img, con[4], -1, (0,0,128), 3)
    return img, finalcontour

def reorder(myPoints):
    #print(myPoints.shape)
    myPointsNew = np.zeros_like(myPoints)
    myPoints = myPoints.reshape((4,2))
    add = myPoints.sum(1)
    myPointsNew[0] = myPoints[np.argmin(add)]
    myPointsNew[3] = myPoints[np.argmax(add)]
    diff = np.diff(myPoints,axis=1)
    myPointsNew[1]= myPoints[np.argmin(diff)]
    myPointsNew[2] = myPoints[np.argmax(diff)]
    return myPointsNew

def warpImg (img,points,w,h,pad=20):
    # print(points)
    points =reorder(points)
    pts1 = np.float32(points)
    pts2 = np.float32([[0,0],[w,0],[0,h],[w,h]])
    matrix = cv2.getPerspectiveTransform(pts1,pts2)
    imgWarp = cv2.warpPerspective(img,matrix,(w,h))
    imgWarp = imgWarp[pad:imgWarp.shape[0]-pad,pad:imgWarp.shape[1]-pad]
    return imgWarp

def findDis(pts1,pts2):
    return ((pts2[0]-pts1[0])**2 + (pts2[1]-pts1[1])**2)**0.5




In [3]:
# img = cv2.imread("/content/real.jpg")
# cv2_imshow(img)

In [ ]:
## Object measurement main.py file.

path = "/content/1.jpg"
img = cv2.imread(path) 

imgcnt, conts = getcontour(img, minarea = 25, filter = 4)
#cv2_imshow(imgcnt)
#print(len(conts))

scale = 3
wp = 210 * scale
hp = 297 * scale

if len(conts) != 0:
    biggest = conts[0][2]
    imgWarp = warpImg(img, biggest, wp,hp)
    #cv2_imshow(imgWarp)
    imgContours2, conts2 = getcontour(imgWarp,minarea=20, filter=4,thr=[100,100],draw = False)
    #cv2_imshow(imgContours2)

    if len(conts) != 0:
        for obj in conts2:
            cv2.polylines(imgContours2,[obj[2]],True,(0,255,0),2)
            #cv2_imshow(imgContours2)
            nPoints = reorder(obj[2])
            nW = round((findDis(nPoints[0][0]//scale,nPoints[1][0]//scale)/10),1)
            nH = round((findDis(nPoints[0][0]//scale,nPoints[2][0]//scale)/10),1)
            cv2.arrowedLine(imgContours2, (nPoints[0][0][0], nPoints[0][0][1]), (nPoints[1][0][0], nPoints[1][0][1]),
                            (255, 0, 255), 3, 8, 0, 0.05)
            cv2.arrowedLine(imgContours2, (nPoints[0][0][0], nPoints[0][0][1]), (nPoints[2][0][0], nPoints[2][0][1]),
                            (255, 0, 255), 3, 8, 0, 0.05)
            x, y, w, h = obj[3]
            cv2.putText(imgContours2, '{}cm'.format(nW), (x + 30, y - 10), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1.5,
                        (255, 0, 255), 2)
            cv2.putText(imgContours2, '{}cm'.format(nH), (x - 70, y + h // 2), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1.5,
                        (255, 0, 255), 2)
    cv2_imshow(imgContours2)